In [96]:
ONE_CAMERA_SEQUENCE_PATH = 'output/2022-04-16-12-15-47_round0/3m/'

In [97]:
import cv2
import numpy as np
import os
from pathlib import Path

### Assoсiate color and depth timestamps

In [88]:
def read_folder(path_to_folder: str):
    """
    Reads all images' paths and timestamps from folder
    :param path_to_folder: path to folder to read
    :return: dictionary where keys are timestamps and values are path to image
    """
    files = os.listdir(path_to_folder)
    timestamps = [float(Path(file).stem) for file in files]
    files = [os.path.join(os.path.basename(path_to_folder), x) for x in files]
    timestamp_image_kvp = dict(zip(timestamps, files))
    return timestamp_image_kvp


def associate(
    color_images: dict, depth_images: dict, offset: float, max_difference: float
) -> list:
    """
    Associates color and depth images
    :param color_images: (timestamp, path) KVP for color images
    :param depth_images: (timestamp, path) KVP for depth images
    :param offset: time offset added to the timestamps of the depth images
    :param max_difference: maximally allowed time difference for matching entries
    :return: best matches for color and depth images
    """
    first_keys = np.asarray(list(color_images.keys()))
    second_keys = np.asarray(list(depth_images.keys()))
    best_matches = list()
    for timestamp in first_keys:
        best_match = second_keys[np.argmin(np.abs(second_keys + offset - timestamp))]
        if abs(best_match + offset - timestamp) < max_difference:
            best_matches.append((timestamp, best_match))
    return sorted(best_matches)


In [102]:
def reformat_names_to_seconds(path):
    names = os.listdir(path)
    names.sort()

    for name in names:
        new_name = name[:6] + '.' + name[6:]

        shutil.move(os.path.join(path, name), 
                    os.path.join(path, new_name))
        
color_path = 'output/2022-04-16-12-15-47_round0/3m/color/'
depth_path = 'output/2022-04-16-12-15-47_round0/3m/depth/'
# reformat_names_to_seconds(color_path)
reformat_names_to_seconds(depth_path)

In [103]:
first_list_M = read_folder(os.path.join(ONE_CAMERA_SEQUENCE_PATH, 'color'))
second_list_M = read_folder(os.path.join(ONE_CAMERA_SEQUENCE_PATH, 'depth'))

matches_M = associate(first_list_M, second_list_M, 0, 1e-3)

with open(os.path.join(ONE_CAMERA_SEQUENCE_PATH, 'associations.txt'), 'w') as f:
    for (a, b) in matches_M:
        f.write('{0} {1} {2} {3}\n'.format(
            a,
            first_list_M[a],
            b,
            second_list_M[b]
        ))

ValueError: could not convert string to float: '.ipynb.'

### Create ORB config

In [100]:
import shutil

K_undist = np.load(os.path.join(ONE_CAMERA_SEQUENCE_PATH, 'calib_params.npy'))

new_config_path = os.path.join(ONE_CAMERA_SEQUENCE_PATH, 'ORB_one_view_config.yaml')
shutil.copyfile('configs/ORB_one_view_config.yaml', new_config_path)

with open(new_config_path, 'r') as f:
    lines[10] = 'Camera1.fx: {}\n'.format(K_undist[0, 0])
    lines[11] = 'Camera1.fy: {}\n'.format(K_undist[1, 1])
    lines[12] = 'Camera1.cx: {}\n'.format(K_undist[0, 2])
    lines[13] = 'Camera1.cy: {}\n'.format(K_undist[1, 2])

with open(new_config_path, 'w') as f:
    f.writelines(lines)